In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

1. Import data
2. Select Test and train data
3. Divide data in target and features
4. Search and Select Relevant Features
5. Fit MultiTarget Classifier
6. Test accuracy and compare
7. Fit MultiTarget Regressor (Try to use only success cases after analysing)
8. Test accuracy and compare
9. Develope ranking method (ART Method)
10. Derive absoluate ranks to check accuracy
11. Predict ranks of test set and find accuracy (Do not calculate but predict)

### 1. Import data

In [2]:
data1900 = pd.read_csv("F:\\ML\\MultiTarget\\data1900.csv")

### 2. Select Test and train data

In [15]:
# Selecting data in test and train
Test  = data1900[data1900.Category=='Test']
Train = data1900[data1900.Category == 'Train']
print("Test Shape:", Test.shape,"\t train shape:", Train.shape)

('Test Shape:', (465, 157), '\t train shape:', (1500, 157))


### 3. Select target and features

In [109]:
# Features starts from 33rd columns
Test_features  = Test.iloc[:,36:]
Train_features = Train.iloc[:,36:]

# Classification targets (Reasoner to be predicted)
Test_Targets_c = Test[filter(lambda x: (x.endswith('CORR')) , Test.columns.tolist())]
Train_Targets_c = Train[filter(lambda x: (x.endswith('CORR')) , Train.columns.tolist())]

for t in Test_Targets_c.columns.tolist():
    Test_Targets_c.loc[Test_Targets_c[t] != 'Correct', t] = 0
    Test_Targets_c.loc[Test_Targets_c[t] == 'Correct', t] = 1
    
    Train_Targets_c.loc[Train_Targets_c[t] != 'Correct', t] = 0
    Train_Targets_c.loc[Train_Targets_c[t] == 'Correct', t] = 1

lbl = preprocessing.LabelEncoder()

for t in Test_features.columns.tolist():
    if(Train_features[t].dtype == 'O'):
        Train_features[t] = lbl.fit_transform(Train_features[t])
        Test_features[t] = lbl.fit_transform(Test_features[t])
'''
for t in Test_Targets_c.columns.tolist():
    if(Train_Targets_c[t].dtype == 'object'):
        Test_Targets_c[t] = lbl.fit_transform(Test_Targets_c[t])
        Train_Targets_c[t] = lbl.fit_transform(Train_Targets_c[t])
    
'''    

#Regressor targets (Execution time of each reasoner)
Test_Targets_r = Test[filter(lambda x: (x.endswith('ETime')) , Test.columns.tolist())]
Train_Targets_r = Train[filter(lambda x: (x.endswith('ETime')) , Train.columns.tolist())]


In [39]:
print(Test_features.shape,Train_Features.shape, Test_Targets_c.shape, Train_Targets_c.shape, Test_Targets_r.shape, Train_Targets_r.shape)

((465, 124), (1500, 124), (465, 10), (1500, 10), (465, 10), (1500, 10))


### 4. Select Relevenat features

### 5. Fit Multilable classifier

In [104]:
clf_rf = RandomForestClassifier(n_estimators=100,random_state=45)
mclf_rf = MultiOutputClassifier(clf_rf, n_jobs=-1)
mclf_rf.fit(Train_features, Train_Targets_c)

In [118]:
mclf_rf.score(Test_features, Test_Targets_c)

0.74838709677419357

In [123]:
m_relevance_vector = mclf_rf.predict(Test_features)

In [117]:
relevance_vector = pd.DataFrame(data = Train_Targets_c, columns = Train_Targets_c.columns.tolist())
for c in  Train_Targets_c.columns.tolist():
    clf_rf.fit(Train_features, Train_Targets_c[c])
    relevance_vector[c] =  clf_rf.predict(Train_features)
    #print(c,':',clf_rf.score(Test_features, Test_Targets_c[c]))
relevance_vector.head()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
0,1,0,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,1,1,1,1
3,1,0,0,0,0,1,0,0,0,0
5,0,0,1,0,0,1,0,1,0,0
6,0,0,1,0,0,1,0,1,0,0


In [124]:
rgs_rf = RandomForestRegressor(n_estimators=100,random_state=45)
mrgs_rf = MultiOutputRegressor(rgs_rf, n_jobs=-1)


In [125]:
mrgs_rf.fit(Train_features, Train_Targets_r)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=45,
           verbose=0, warm_start=False),
           n_jobs=-1)

In [126]:
mrgs_rf.score(Test_features, Test_Targets_r)

0.70094398753695764

In [128]:
m_timerank_vector = mrgs_rf.predict(Test_features)

In [130]:
timerank_vector = pd.DataFrame(data = Train_Targets_r, columns = Train_Targets_r.columns.tolist())
for c in  Train_Targets_r.columns.tolist():
    rgs_rf.fit(Train_features, Train_Targets_r[c])
    timerank_vector[c] =  rgs_rf.predict(Train_features)
    #print(c,':',clf_rf.score(Test_features, Test_Targets_c[c]))
timerank_vector.head()

,Racer_ETime,ELepHant_ETime,TrOWL_ETime,JFact_ETime,MOReHermiT_ETime,Konclude_ETime,FaCT++_ETime,ELK_ETime,Pellet-OA4_ETime,HermiT-OA4_ETime
0,74953.771667,36.210000,5234.950000,135538.04500,2995.300000,3858.708333,17558.080000,1725.930000,8253.410000,14318.990000
1,4644.003333,13.050000,1500.630000,6633.27000,1649.195000,389.990000,2949.203667,859.973333,2405.470000,2495.950000
3,27288.220000,22.430000,4708.596667,40777.08000,47703.430000,1207.210000,39842.053333,456.650000,37450.360000,33430.180000
5,207627.850000,679.201417,5897.824417,204817.89725,203779.115583,4026.699500,65112.786250,2221.227417,209744.002417,177813.587167
6,207627.850000,679.201417,5897.824417,204817.89725,203779.115583,4026.699500,65112.786250,2221.227417,209744.002417,177813.587167
